In [1]:
#Importing Packages
import pandas as pd
import numpy as np
import lxml
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
#Getting States Data
states = {}
url = "https://ngodarpan.gov.in/index.php/home/statewise"
source = requests.get(url).text
soup = BeautifulSoup(source,"lxml")
content = soup.find("table", class_ = "middle_section_bg")
con = content.find_all("a")
for c in con:
    d = c.text.replace("\xa0","").split("(")[0]
    states[d] = [c.text.split("(")[1].replace(")",""), c['href'][:-1] + "{}"]

In [3]:
states = {'LAKSHADWEEP': ['2',  'https://ngodarpan.gov.in/index.php/home/statewise_ngo/2/31/{}']}

In [4]:
for k,v in states.items():
    num = int(v[0])//10
    url = v[1]
    data = pd.DataFrame(columns = ["S.no.","ID","Name","Email-id","Mobile no.","Address","City","State","Sector","Website"])
    data.drop(data.index, inplace=True)
    count = 0

    for j in range(1,num + 2):
        chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"
        driver = webdriver.Chrome(chromedriver)
        driver.get(url.format(j))

        for i in range(1,11):
            try:
                path = "/html/body/div[9]/div[1]/div[3]/div/div/div[2]/table/tbody/tr[{}]/td[2]/a"
                time.sleep(5)
                driver.find_element_by_xpath(path.format(i)).click()
                WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CLASS_NAME,"modal-content")))
                time.sleep(5)

                html = driver.page_source
                soup = BeautifulSoup(html,"lxml")
                count = count + 1
                content = soup.find("div", {"id":"ngo_info_modal", "class":"modal fade in"})
                mainbody = content.find("div", class_ = "container")   
                name = mainbody.find("h3").text.strip()
                ids = mainbody.table.find("span", {"id":"UniqueID"}).text.strip()
                tables = mainbody.find_all("table")
                sector = tables[3].find("td", {"id":"key_issues"}).text.strip().replace("&amp;","&").upper()
                address = tables[7].find("td", {"id":"address"}).text.strip()
                city = tables[7].find("td", {"id":"city"}).text.strip()
                state = tables[7].find("td", {"id":"state_p_ngo"}).text.strip()
                mobile = tables[7].find("td", {"id":"mobile_n"}).text.strip()
                website = tables[7].find("td", {"id":"ngo_web_url"}).text.strip()
                email = tables[7].find("td", {"id":"email_n"}).text.strip().replace("(at)","@").replace("[dot]",".")
                data = data.append({"S.no.":count,"ID":ids,"Name":name,"Email-id":email,"Mobile no.":mobile,"Address":address,"City":city,"State":state,"Sector":sector,"Website":website},ignore_index = True)

                x = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="ngo_info_modal"]/div[2]/div/div[1]/button')))
                driver.execute_script("arguments[0].click();", x)
                WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CLASS_NAME,"modal-content")))
            except Exception as e:
                pass
        driver.close()

    data.loc[data["Sector"].str.contains("EDUCATION"), "Sector"] = "Education"
    data.loc[data["Sector"].str.contains("HEALTH"), "Sector"] = "Health"
    data.loc[data["Sector"].str.contains("EMPOWERMENT"), "Sector"] = "Empowerment"
    data.loc[~data["Sector"].str.contains("Education|Health|Empowerment"), "Sector"] = "Other"
    p = r"C:\Users\jaske\OneDrive\Desktop\data"
    path = p + "\\" + k  + ".xlsx"
    data.to_excel(path, index =False)